In [ ]:
import time
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF

In [ ]:
topk = 10
rank_list = [i+1 for i in range(topk)]
latent = 50

In [ ]:
%%time
ML100K_URL = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.data'
dataset = pd.read_csv(
  ML100K_URL, 
  names=["user_id", "item_id", "rating", "timestamp"], 
  sep="\t"
)
matrix_data = csr_matrix(
  (dataset.rating, 
  (dataset.user_id, dataset.item_id))
)

CPU times: user 49.2 ms, sys: 15 ms, total: 64.2 ms
Wall time: 332 ms


In [ ]:
%%time
nmf = NMF(n_components=latent)
# ユーザ因子行列
W = nmf.fit_transform(matrix_data)
# アイテム因子行列
H = nmf.components_

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,


CPU times: user 3.72 s, sys: 330 ms, total: 4.05 s
Wall time: 3.82 s


/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1641: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  ConvergenceWarning,


In [ ]:
print(W.shape)
print(H.shape)

(944, 50)
(50, 1683)


In [ ]:
# ユーザ因子行列とアイテム因子行列を掛け合わせ、評価行列を復元する
WH = np.dot(W, H)

In [ ]:
WH[1:5, 1:5]

array([[4.70419190e+00, 1.93611361e+00, 9.50869790e-01, 3.91134488e+00],
       [2.21065484e+00, 4.33255954e-04, 1.32081646e-01, 1.09551219e-01],
       [6.93788234e-01, 1.85656584e-02, 4.01470056e-03, 6.10535140e-02],
       [6.27698643e-01, 1.18935802e-02, 5.02208601e-02, 1.60362136e-01]])

In [ ]:
# すでに評価したユーザ-アイテムの組み合わせを取り除く
recommend_matrix = np.where(
  matrix_data.toarray(), 0 ,WH
)

In [ ]:
matrix_data.toarray()[1:5, 1:5]

array([[5, 3, 4, 3],
       [4, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=int64)

In [ ]:
recommend_matrix[1:5, 1:5]

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 4.33255954e-04, 1.32081646e-01, 1.09551219e-01],
       [6.93788234e-01, 1.85656584e-02, 4.01470056e-03, 6.10535140e-02],
       [6.27698643e-01, 1.18935802e-02, 5.02208601e-02, 1.60362136e-01]])

In [ ]:
%%time
# ユニークなユーザIDのリストを作成する
uq_users = np.sort(dataset.user_id.unique().tolist())

# 推薦結果を格納するテーブルを作成する
df_recommend_list = pd.DataFrame(
  columns=[
    'user_id', #ユーザID
    'item_id', #アイテムID
    'score',   #スコア
    'rank'     #順位
  ]
)

# 各ユーザに対して、トップ10アイテムを絞り込む
for user_id in uq_users:

  # 対象ユーザの未接触アイテムへのスコア情報を取得する
  item_scores = recommend_matrix[user_id]

  df_recommend = pd.DataFrame()
  df_recommend['user_id'] = [user_id] * topk
  # トップ10アイテムのアイテムIDを取得する
  df_recommend['item_id'] = \
    np.argsort(item_scores)[::-1][:topk]
  # トップ10アイテムのスコアを取得する
  df_recommend['score'] = \
    np.sort(item_scores)[::-1][:topk]
  df_recommend["rank"] = rank_list

  # 全体テーブルに格納する
  df_recommend_list = \
    df_recommend_list.append(
        df_recommend, 
        ignore_index=True
    )

CPU times: user 4.59 s, sys: 30.9 ms, total: 4.62 s
Wall time: 5.73 s


In [ ]:
df_recommend_list

,user_id,item_id,score,rank
0,1,285,4.216171,1
1,1,408,3.789276,2
2,1,531,3.524720,3
3,1,655,3.517579,4
4,1,462,3.235171,5
...,...,...,...,...
9425,943,17,2.728399,6
9426,943,265,2.661973,7
9427,943,357,2.650266,8
9428,943,156,2.644612,9
